In [ ]:
"""
Aggregate multiple raster datasets to ward boundaries using rasterstats
Purpose: Calculate mean values of raster data within ward polygons for regression analysis
"""

In [2]:
import os
os.getcwd()
os.chdir("G:/OneDrive/casa0010dissertation/00_06 jaipur code_mgwr")

In [4]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.enums import Resampling
from rasterstats import zonal_stats
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

def resample_raster_to_100m(input_path, output_path, target_resolution=100.0):
    """
    Resample raster to 100m resolution using bilinear interpolation

    Args:
        input_path (str): Path to input raster file
        output_path (str): Path to output resampled raster file
        target_resolution (float): Target resolution in meters (default: 100.0)

    Returns:
        bool: True if successful, False otherwise
    """
    try:
        with rasterio.open(input_path) as src:
            current_res_x, current_res_y = abs(src.res[0]), abs(src.res[1])

            if abs(current_res_x - target_resolution) < 1 and abs(current_res_y - target_resolution) < 1:
                print(f"  Already at ~{target_resolution}m resolution, copying file...")
                with rasterio.open(output_path, 'w', **src.profile) as dst:
                    dst.write(src.read())
                return True

            transform, width, height = calculate_default_transform(
                src.crs, src.crs, src.width, src.height, *src.bounds,
                resolution=(target_resolution, target_resolution)
            )

            profile = src.profile.copy()
            profile.update({
                'transform': transform,
                'width': width,
                'height': height,
                'compress': 'lzw'
            })

            print(f"  Resampling from {current_res_x:.1f}m to {target_resolution}m...")
            print(f"  New dimensions: {width} x {height}")

            with rasterio.open(output_path, 'w', **profile) as dst:
                for i in range(src.count):
                    reproject(
                        source=rasterio.band(src, i + 1),
                        destination=rasterio.band(dst, i + 1),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.bilinear
                    )

            print(f"  ✅ Successfully resampled to: {output_path}")
            return True

    except Exception as e:
        print(f"  ❌ Error resampling {input_path}: {str(e)}")
        return False

def resample_all_rasters(raster_config, target_resolution=100.0):
    """
    Resample all rasters to target resolution

    Args:
        raster_config (dict): Dictionary of {input_path: column_name}
        target_resolution (float): Target resolution in meters

    Returns:
        dict: Updated raster config with resampled file paths
    """
    resampled_dir = Path("data/cleaned/resampled_100m")
    resampled_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n{'='*60}")
    print(f"RESAMPLING RASTERS TO {target_resolution}m RESOLUTION")
    print(f"{'='*60}")

    resampled_config = {}

    for input_path, column_name in raster_config.items():
        input_file = Path(input_path)
        output_file = resampled_dir / f"{input_file.stem}_100m{input_file.suffix}"

        print(f"\nProcessing: {input_file.name}")

        if resample_raster_to_100m(input_path, str(output_file), target_resolution):
            resampled_config[str(output_file)] = column_name
        else:
            print(f"  ⚠️  Failed to resample, using original file")
            resampled_config[input_path] = column_name

    print(f"\n✅ Resampling completed!")
    print(f"Resampled files saved to: {resampled_dir}")

    return resampled_config

def check_raster_info(raster_path):
    """
    Check and display basic information about a raster file

    Args:
        raster_path (str): Path to the raster file

    Returns:
        dict: Raster metadata information
    """
    with rasterio.open(raster_path) as src:
        info = {
            'crs': src.crs,
            'shape': src.shape,
            'resolution': src.res,
            'bounds': src.bounds,
            'dtype': src.dtypes[0],
            'nodata': src.nodata
        }
        print(f"Raster: {Path(raster_path).name}")
        print(f"  CRS: {info['crs']}")
        print(f"  Shape: {info['shape']}")
        print(f"  Resolution: {info['resolution']}")
        print(f"  Data type: {info['dtype']}")
        print(f"  NoData value: {info['nodata']}")
        print("-" * 50)

    return info

# 1: Updated the function signature and logic to handle different stats per raster.
def aggregate_raster_to_wards(ward_gdf, raster_paths, output_columns, stats_dict):
    """
    Aggregate multiple raster datasets to ward boundaries using zonal statistics.

    Args:
        ward_gdf (GeoDataFrame): Ward boundary polygons.
        raster_paths (list): List of paths to raster files.
        output_columns (list): List of output column names for each raster.
        stats_dict (dict): A dictionary mapping column names to the statistic to calculate
                           (e.g., {'col1': 'sum', 'col2': 'mean'}). Defaults to 'mean' if a
                           column is not in the dictionary.

    Returns:
        GeoDataFrame: Ward boundaries with aggregated raster values.
    """
    result_gdf = ward_gdf.copy()

    if result_gdf.crs != 'EPSG:32643':
        print(f"Warning: Ward CRS is {result_gdf.crs}, reprojecting to EPSG:32643")
        result_gdf = result_gdf.to_crs('EPSG:32643')

    print(f"Processing {len(raster_paths)} raster files...")
    print(f"Ward boundaries: {len(result_gdf)} features")
    print("=" * 60)

    for i, (raster_path, column_name) in enumerate(zip(raster_paths, output_columns)):
        print(f"\nProcessing raster {i+1}/{len(raster_paths)}: {Path(raster_path).name}")

        try:
            raster_info = check_raster_info(raster_path)

            # Get the statistic for the current column, default to 'mean' if not specified.
            stat_to_calc = stats_dict.get(column_name, 'mean')

            print(f"Calculating zonal statistics for {column_name} with statistic: '{stat_to_calc}'...")

            # Perform zonal statistics with the specified statistic.
            zs = zonal_stats(
                result_gdf.geometry,
                raster_path,
                stats=[stat_to_calc],  # zonal_stats expects a list of stats.
                nodata=raster_info['nodata'],
                all_touched=True,
                geojson_out=False
            )

            # Extract the calculated values. The key in the result dict will be the stat we calculated.
            values = [feature[stat_to_calc] if feature and feature[stat_to_calc] is not None else np.nan
                      for feature in zs]

            result_gdf[column_name] = values

            # Print summary statistics.
            valid_values = [v for v in values if not np.isnan(v)]
            print(f"  Successfully processed: {len(valid_values)}/{len(values)} wards")
            if valid_values:
                print(f"  Summary of calculated '{stat_to_calc}' values:")
                print(f"    Min: {np.min(valid_values):.4f}")
                print(f"    Max: {np.max(valid_values):.4f}")
                print(f"    Mean: {np.mean(valid_values):.4f}")

        except Exception as e:
            print(f"Error processing {raster_path}: {str(e)}")
            result_gdf[column_name] = np.nan

    return result_gdf

def main():
    """
    Main function to resample rasters to 100m resolution and aggregate to ward boundaries
    """
    WARD_FILE = "data/raw/JMC_all_v2.geojson"

    raster_config = {
        "data/cleaned/Illuminated_Residential_Volume.tif": "illum_vol",
        "data/cleaned/Residential_Light_Intensity_10m.tif": "light_intensity",
        "data/cleaned/combined_poi_kde_1600m_residential.tif": "poi_kde"
    }

    OUTPUT_DIR = Path("data/cleaned")
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_FILE = OUTPUT_DIR / "ward_aggregated_data_100m.geojson"

    print("=" * 60)
    print("RASTER RESAMPLING & WARD AGGREGATION TOOL")
    print("Step 1: Resample all rasters to 100m resolution")
    print("Step 2: Aggregate resampled rasters to ward boundaries")
    print("Processing 3 variables: illum_vol, light_intensity, poi_kde")
    print("=" * 60)

    try:
        print("\nChecking original raster file availability...")
        missing_files = []
        for raster_path in raster_config.keys():
            if not Path(raster_path).exists():
                missing_files.append(raster_path)
                print(f"  ❌ Missing: {raster_path}")
            else:
                print(f"  ✅ Found: {Path(raster_path).name}")

        if missing_files:
            print(f"\nError: {len(missing_files)} raster files are missing!")
            print("Please check the file paths and try again.")
            return None

        resampled_config = resample_all_rasters(raster_config, target_resolution=100.0)
        resampled_paths = list(resampled_config.keys())
        output_columns = list(resampled_config.values())

        print(f"\n{'='*60}")
        print("LOADING WARD BOUNDARIES")
        print(f"{'='*60}")
        print(f"Loading ward boundaries from: {WARD_FILE}")
        ward_gdf = gpd.read_file(WARD_FILE)
        print(f"Loaded {len(ward_gdf)} ward polygons")
        print(f"Ward CRS: {ward_gdf.crs}")

        print(f"\n{'='*60}")
        print("AGGREGATING RESAMPLED RASTERS TO WARDS")
        print(f"{'='*60}")

        #2: Define a dictionary to specify the aggregation method for each variable.
        stats_to_calculate = {
            "illum_vol": "sum",          # Use 'sum' for illum_vol
            "light_intensity": "mean",   # Use 'mean' for light_intensity
            "poi_kde": "mean"            # Use 'mean' for poi_kde
        }

        #3: Pass the new stats dictionary to the aggregation function.
        result_gdf = aggregate_raster_to_wards(
            ward_gdf=ward_gdf,
            raster_paths=resampled_paths,
            output_columns=output_columns,
            stats_dict=stats_to_calculate
        )

        print("\n" + "=" * 60)
        print("AGGREGATION RESULTS SUMMARY")
        print("=" * 60)

        for col in output_columns:
            if col in result_gdf.columns:
                valid_count = result_gdf[col].notna().sum()
                total_count = len(result_gdf)

                print(f"\n{col}:")
                print(f"  Valid values: {valid_count}/{total_count} wards")

                if valid_count > 0:
                    values = result_gdf[col].dropna()
                    print(f"  Min: {values.min():.4f}")
                    print(f"  Max: {values.max():.4f}")
                    print(f"  Mean: {values.mean():.4f}")
                    print(f"  Std: {values.std():.4f}")

        print(f"\nSaving results to: {OUTPUT_FILE}")
        result_gdf.to_file(OUTPUT_FILE, driver='GeoJSON')

        csv_output = OUTPUT_DIR / "ward_aggregated_data_100m.csv"
        result_df = pd.DataFrame(result_gdf.drop('geometry', axis=1))
        result_df.to_csv(csv_output, index=False)
        print(f"Data table saved to: {csv_output}")

        print(f"\nCalculating correlation matrix...")
        correlation_matrix = result_gdf[output_columns].corr().round(3)
        corr_output = OUTPUT_DIR / "correlation_matrix_100m.csv"
        correlation_matrix.to_csv(corr_output, index=True)
        print(f"Correlation matrix saved to: {corr_output}")

        print("\n✅ Processing completed successfully!")
        print(f"Output files:")
        print(f"  - GeoJSON: {OUTPUT_FILE}")
        print(f"  - CSV data: {csv_output}")
        print(f"  - Correlation matrix: {corr_output}")
        print(f"  - Resampled rasters: data/cleaned/resampled_100m/")

        print("\nFirst 5 rows of aggregated data:")
        print(result_gdf[output_columns].head())

        print(f"\nCorrelation Matrix (100m resolution):")
        print(correlation_matrix)

        return result_gdf

    except Exception as e:
        print(f"\nError during processing: {str(e)}")
        print("Please check your input files and try again.")
        return None

# 4: Updated the quick_aggregate function to be consistent with the new changes.
def quick_aggregate(ward_gdf, raster_dict, stats_dict):
    """
    Quick aggregation function when you already have loaded data

    Args:
        ward_gdf (GeoDataFrame): Ward boundaries
        raster_dict (dict): Dictionary with {raster_path: column_name}
        stats_dict (dict): Dictionary mapping column names to statistics (e.g., {'col1': 'sum'})

    Returns:
        GeoDataFrame: Aggregated results

    Example:
        raster_dict = {
            "path/to/illum_vol.tif": "illum_vol",
            "path/to/light_intensity.tif": "light_intensity"
        }
        stats_to_calc = {"illum_vol": "sum", "light_intensity": "mean"}
        result = quick_aggregate(ward_gdf, raster_dict, stats_to_calc)
    """

    raster_paths = list(raster_dict.keys())
    output_columns = list(raster_dict.values())

    return aggregate_raster_to_wards(
        ward_gdf=ward_gdf,
        raster_paths=raster_paths,
        output_columns=output_columns,
        stats_dict=stats_dict
    )

if __name__ == "__main__":
    result = main()

    if result is not None:
        print("\n" + "=" * 60)
        print("CORRELATION MATRIX (100m resolution - for regression analysis)")
        print("=" * 60)

        numeric_cols = ['illum_vol', 'light_intensity', 'poi_kde']
        available_cols = [col for col in numeric_cols if col in result.columns]

        if len(available_cols) > 1:
            correlation_matrix = result[available_cols].corr()
            print(correlation_matrix.round(3))
        else:
            print("Not enough numeric columns for correlation analysis")

RASTER RESAMPLING & WARD AGGREGATION TOOL
Step 1: Resample all rasters to 100m resolution
Step 2: Aggregate resampled rasters to ward boundaries
Processing 3 variables: illum_vol, light_intensity, poi_kde

Checking original raster file availability...
  ✅ Found: Illuminated_Residential_Volume.tif
  ✅ Found: Residential_Light_Intensity_10m.tif
  ✅ Found: combined_poi_kde_1600m_residential.tif

RESAMPLING RASTERS TO 100.0m RESOLUTION

Processing: Illuminated_Residential_Volume.tif
  Resampling from 10.0m to 100.0m...
  New dimensions: 223 x 281
  ✅ Successfully resampled to: data\cleaned\resampled_100m\Illuminated_Residential_Volume_100m.tif

Processing: Residential_Light_Intensity_10m.tif
  Resampling from 10.0m to 100.0m...
  New dimensions: 223 x 281
  ✅ Successfully resampled to: data\cleaned\resampled_100m\Residential_Light_Intensity_10m_100m.tif

Processing: combined_poi_kde_1600m_residential.tif
  Already at ~100.0m resolution, copying file...

✅ Resampling completed!
Resampled fi

In [5]:
ward_mean =gpd.read_file("data/cleaned/ward_aggregated_data_100m.geojson")

In [6]:
ward_mean.head(5)

,Id,Area,Ward_No,POP,DENS_PPH,NAME,assembly,jmc,X,Y,lon,lat,illum_vol,light_intensity,poi_kde,geometry
0,1,502,1,11930,24,VIDHYADHAR NAGAR,VidyadharNgr_Assembly,150,574626.431817,2.988687e+06,75.752277,27.018317,54574.414062,7.748671,1.659231e-10,"MULTIPOLYGON (((574011.047 2987648.778, 572694..."
1,2,477,2,11096,23,VIDHYADHAR NAGAR,VidyadharNgr_Assembly,150,574052.087406,2.987101e+06,75.746393,27.004035,101310.851562,16.355628,4.443493e-10,"MULTIPOLYGON (((574011.047 2987648.778, 575313..."
2,3,150,3,12110,81,VIDHYADHAR NAGAR,VidyadharNgr_Assembly,150,575995.451217,2.987597e+06,75.766009,27.008404,86250.023438,17.579170,3.736982e-10,"MULTIPOLYGON (((575313.015 2987671.726, 575931..."
3,4,156,4,12097,78,VIDHYADHAR NAGAR,VidyadharNgr_Assembly,150,576790.537250,2.987811e+06,75.774036,27.010297,81452.664062,16.990320,2.886297e-10,"MULTIPOLYGON (((576455.893 2986652.841, 576416..."
4,5,453,5,12158,27,VIDHYADHAR NAGAR,VidyadharNgr_Assembly,150,578926.085803,2.987224e+06,75.795523,27.004871,200711.187500,13.872617,1.640855e-10,"MULTIPOLYGON (((577342.574 2987267.38, 577350...."


In [23]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant
#
# X = ward_mean[['illum_vol', 'light_intensity', 'poi_kde']]
# X = add_constant(X)
#
# vif = pd.DataFrame()
# vif['Variable'] = X.columns
# vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# print(vif)


          Variable        VIF
0            const  20.278956
1        illum_vol   2.117307
2  light_intensity   2.285745
3          poi_kde   2.447747


In [10]:
import geopandas as gpd

In [11]:
gdf_t=gpd.read_file("data/cleaned/ward_aggregated_data_100m.geojson")

In [12]:
gdf_t.columns

Index(['Id', 'Area', 'Ward_No', 'POP', 'DENS_PPH', 'NAME', 'assembly', 'jmc',
       'X', 'Y', 'lon', 'lat', 'illum_vol', 'light_intensity', 'poi_kde',
       'geometry'],
      dtype='object')

In [16]:
gdf_t['poi_kde'].describe()

count    2.510000e+02
mean     4.075008e-09
std      2.968007e-09
min      1.640855e-10
25%      1.795912e-09
50%      3.271182e-09
75%      5.597422e-09
max      1.308537e-08
Name: poi_kde, dtype: float64

In [17]:
scaling_factor = 1e9  # 1,000,000,000

#  poi_kde is too small, scale it up for later MGWR analysis
gdf_t['poi_kde_scaled'] = gdf_t['poi_kde'] * scaling_factor

print("\n--- POI KDE distribution AFTER scaling ---")
print(gdf_t['poi_kde_scaled'].describe())


--- POI KDE distribution AFTER scaling ---
count    251.000000
mean       4.075008
std        2.968007
min        0.164086
25%        1.795912
50%        3.271182
75%        5.597422
max       13.085366
Name: poi_kde_scaled, dtype: float64


In [18]:
gdf_t.columns

Index(['Id', 'Area', 'Ward_No', 'POP', 'DENS_PPH', 'NAME', 'assembly', 'jmc',
       'X', 'Y', 'lon', 'lat', 'illum_vol', 'light_intensity', 'poi_kde',
       'geometry', 'poi_kde_scaled'],
      dtype='object')

In [19]:
gdf_t.rename(columns={'illum_vol': 'illum_vol_density'}, inplace=True)


In [20]:
gdf_t.columns

Index(['Id', 'Area', 'Ward_No', 'POP', 'DENS_PPH', 'NAME', 'assembly', 'jmc',
       'X', 'Y', 'lon', 'lat', 'illum_vol_density', 'light_intensity',
       'poi_kde', 'geometry', 'poi_kde_scaled'],
      dtype='object')

In [22]:
gdf_t.to_file("data/cleaned/ward_aggregated_data_100m_poi.geojson")

In [3]:
import pandas as pd
import geopandas as gpd

In [4]:
gdf2 = gpd.read_file("data/cleaned/ward_aggregated_data_100m_poi.geojson")
# Calculate total population
pop_sum = gdf2['POP'].sum()
print(f"Total population: {pop_sum:,.0f}")

Total population: 3,053,908
